In [ ]:
#Notebook description
#This notebook is used to compare a company amongs its subindustry constituents

In [ ]:
#import libraries
import pandas as pd
import yfinance as yf
import requests
import time
import json
import os
import datetime
from quickfs import QuickFS
from dotenv import load_dotenv
import sys
sys.path.append(r"e:\Coding Projects\Investment Analysis")
from Quantapp.EconomicData import EconomicData
qe = EconomicData()
load_dotenv()
#from edgar import *
#set_identity("Joey blum joeyblum@indigo.com")



In [ ]:
#create client to query quickfs
#set api key for quickfs
api_key = '72418c68701409373b2902e70c58096c0c9d827e'
os.environ['API_QUICKFS'] = api_key

#create quickfs client
api_key = os.environ['API_QUICKFS']
client = QuickFS(api_key)


In [ ]:
#set parameters

ticker = 'AAPL'
should_update = False #careful with this one, it will update all the data and use up you credits. set to True if you want to force an update on the data

sp500_companies = qe.retrieve_market_tables()['SP500_TABLE']

#retrieve sector
sector = sp500_companies[sp500_companies['Symbol'] == ticker]['Sector'].values[0]

#retrieve subindustry
subindustry = sp500_companies[sp500_companies['Symbol'] == ticker]['Sub-Industry'].values[0]

all_companies = list(sp500_companies['Symbol'])
all_companies_query = [x + ':US' for x in all_companies]

#retreive all symbols in same sector
sector_companies = list(sp500_companies[sp500_companies['Sector'] == sector]['Symbol'])
sector_companies_query = [x + ':US' for x in sector_companies]

#retreive all symbols in same subindustry
subindustry_companies = list(sp500_companies[sp500_companies['Sub-Industry'] == subindustry]['Symbol'])
subindustry_companies_query = [x + ':US' for x in subindustry_companies]

print('Sector:', sector)
print('Subindustry:', subindustry)
print('Companies in sector:', sector_companies)
print('Companies in subindustry:', subindustry_companies)



In [ ]:
#define custom functions

#convinence functions for working with dates
#------------------------------------------------------------
def simplify_datetime_index(series):
    """
    Simplifies the DateTime index of a Series to contain only the date (YYYY-MM-DD),
    maintaining it as a DateTimeIndex without timezone information.
    
    Parameters:
        series (pd.Series): The input Series with a DateTimeIndex.
    
    Returns:
        pd.Series: The Series with the DateTime index simplified to YYYY-MM-DD.
    """
    if not isinstance(series.index, pd.DatetimeIndex):
        raise TypeError("The Series index must be a DateTimeIndex.")
    
    # Remove timezone information if present
    if series.index.tz is not None:
        series = series.copy()
        series.index = series.index.tz_convert('UTC').tz_localize(None)
    
    # Normalize the index to remove the time component
    series.index = series.index.normalize()
    
    return series
#flattens date from YYYY-MM-DD to YYYYMMDD
def flatten_date(date_str):
    date_obj = datetime.datetime.strptime(date_str, "%Y-%m-%d")
    return date_obj.strftime("%Y%m%d")
#------------------------------------------------------------





#Call to API
#------------------------------------------------------------
#function to get all available metrics in QuickFS
def get_metrics(client):
    """
    Retrieves the metrics available from the QuickFS API.
    
    Parameters:
        client (QuickFSClient): The client object used to interact with the API.
    
    Returns:
        list: A list of dictionaries containing the available metrics and their descriptions.
    """
    # Retrieve the metrics available from the API
    metrics = client.get_available_metrics()
    
    # Return the list of metrics
    return metrics

def get_latest_earnings_date(ticker):
    earnings_dates = yf.Ticker(ticker).earnings_dates.dropna().index
    last_earnings_date = str(earnings_dates[0].strftime('%Y-%m-%d'))
    return last_earnings_date

def get_companies_updated_on_or_after_date(client, date):
    flatten_date = flatten_date(date)
    updated_companies = client.get_updated_companies(country='US', date=flatten_date)
    return updated_companies

def retrieve_data_from_API(ticker_string,client):
    print('Querying data API!')
    #convert dictionary to 
    data = client.get_data_full(symbol=ticker_string)
    data = process_full_data(data, client)
    print('Data retreived successfully!\n')
    save_company_info(ticker_string, data['financials_annual'], data['financials_quarterly'], data['metadata'])
    print('Data saved successfully!\n')
    return data

def retrieve_data(ticker_string, data_type='annual', statement_type='income_statement', should_update=False, client=client):
    # Check if the ticker folder exists; if not, create it
    parent_folder_name = 'company_data'
    ticker_folder = os.path.join(parent_folder_name, ticker_string)
    if not os.path.exists(ticker_folder):
        os.makedirs(ticker_folder)
        print('Company folder does not exist, creating it now!\n')
    else:
        print('Company folder exists\n')

    # Check if the company folder is empty
    if len(os.listdir(ticker_folder)) == 0:
        print(ticker_folder)
        print('No data found for this company\n')
        retrieve_data_from_API(ticker_string, client)
    else:
        print('Data found for this company\n')
        
    if should_update:
        print('Manually updating data\n')
        retrieve_data_from_API(ticker_string, client)
    else:
        print('Set should_update to True to manually update data\n')
        
    # Check if the data is up to date; if not, retrieve it from the API
    # (This comment is a reminder for future implementation if needed.)

    # If the data exists and is up to date, load it from disk according to the fiscal period
    if data_type == 'annual':
        print('Retrieving annual data\n')
        return pd.read_csv(os.path.join(ticker_folder, data_type, statement_type + '.csv'))
    elif data_type == 'quarterly':
        print('Retrieving quarterly data\n')
        return pd.read_csv(os.path.join(ticker_folder, data_type, statement_type + '.csv'))
    elif data_type == 'metadata':
        print('Retrieving metadata\n')
        with open(os.path.join(ticker_folder, 'metadata.json')) as f:
            return json.load(f)
    else:
        print('Invalid fiscal period\n')

def retrieve_data_for_companies(companies, data_type='annual', statement_type='income_statement', should_update=False, client=client):
    data = {}
    for company in companies:
        ticker = company.split(':')[0]
        print(f'Retrieving data for {ticker}')
        data[ticker] = retrieve_data(ticker, data_type, statement_type, should_update, client)
    
    #print usage of data
    print(client.get_usage())
    return data
        
#------------------------------------------------------------



#Procesing data once it is retreived

#------------------------------------------------------------
def process_full_data(full_data, client):
    """
    Processes the full_data dictionary to extract company metadata and financials.
    It converts financial data (annual and quarterly) into DataFrames with sorted columns,
    and then separates them into dictionaries based on statement types using available metrics.
    
    Parameters:
        full_data (dict): Dictionary containing keys 'metadata' and 'financials' (with keys 'annual' and 'quarterly').
        client: Parameter used to retrieve metrics via get_metrics(client).
    
    Returns:
        dict: A dictionary containing:
            'ticker': company ticker,
            'metadata': company metadata,
            'financials_annual': dictionary of annual financial DataFrames,
            'financials_quarterly': dictionary of quarterly financial DataFrames.
    """
    import pandas as pd

    def create_financials_dict(financials_df, client):
        # Retrieve the metrics from the client using get_metrics
        
        metrics = get_metrics(client)
        # Identify metrics that are not present in the DataFrame
        metrics_not_in_df = []
        for field in metrics:
            if field['metric'] not in financials_df.columns:
                metrics_not_in_df.append(field['metric'])

        # Filter out metrics not in the DataFrame and sort the remaining metrics
        metrics = [field for field in metrics if field['metric'] not in metrics_not_in_df]
        metrics = sorted(metrics, key=lambda x: x['metric'])
        

        
        
        # Extract metrics based on statement type
        metrics_income_statement    = [field['metric'] for field in metrics if field['statement_type'] == 'income_statement']
        metrics_balance_sheet       = [field['metric'] for field in metrics if field['statement_type'] == 'balance_sheet']
        metrics_cash_flow_statement = [field['metric'] for field in metrics if field['statement_type'] == 'cash_flow_statement']
        metrics_computed             = [field['metric'] for field in metrics if field['statement_type'] == 'computed']
        metrics_misc                = [field['metric'] for field in metrics if field['statement_type'] == 'misc']
  
        # Create separate DataFrames for each statement type by concatenating metrics_misc with the respective group
        income_statement_df     = pd.concat([financials_df[metrics_misc], financials_df[metrics_income_statement]], axis=1)
        balance_sheet_df        = pd.concat([financials_df[metrics_misc], financials_df[metrics_balance_sheet]], axis=1)
        cash_flow_statement_df  = pd.concat([financials_df[metrics_misc], financials_df[metrics_cash_flow_statement]], axis=1)
        computed_df              = pd.concat([financials_df[metrics_misc], financials_df[metrics_computed]], axis=1)
        misc_df                 = financials_df[metrics_misc]
        
        #fiscal_year_key
        #fiscal_year_number
        #fiscal_quarter_key
        #fiscal_quarter_number 
        
        #check if fiscal_year_key, fiscal_year_number, fiscal_quarter_key, fiscal_quarter_number are in the financials_df
        #if they are, add columns to each dataframe
        
        if 'fiscal_year_key' in financials_df.columns:
            income_statement_df['fiscal_year_key'] = financials_df['fiscal_year_key']
            balance_sheet_df['fiscal_year_key'] = financials_df['fiscal_year_key']
            cash_flow_statement_df['fiscal_year_key'] = financials_df['fiscal_year_key']
            computed_df['fiscal_year_key'] = financials_df['fiscal_year_key']
            misc_df['fiscal_year_key'] = financials_df['fiscal_year_key']
            
        if 'fiscal_year_number' in financials_df.columns:
            income_statement_df['fiscal_year_number'] = financials_df['fiscal_year_number']
            balance_sheet_df['fiscal_year_number'] = financials_df['fiscal_year_number']
            cash_flow_statement_df['fiscal_year_number'] = financials_df['fiscal_year_number']
            computed_df['fiscal_year_number'] = financials_df['fiscal_year_number']
            misc_df['fiscal_year_number'] = financials_df['fiscal_year_number']
            
        if 'fiscal_quarter_key' in financials_df.columns:
            income_statement_df['fiscal_quarter_key'] = financials_df['fiscal_quarter_key']
            balance_sheet_df['fiscal_quarter_key'] = financials_df['fiscal_quarter_key']
            cash_flow_statement_df['fiscal_quarter_key'] = financials_df['fiscal_quarter_key']
            computed_df['fiscal_quarter_key'] = financials_df['fiscal_quarter_key']
            misc_df['fiscal_quarter_key'] = financials_df['fiscal_quarter_key']
            
        if 'fiscal_quarter_number' in financials_df.columns:
            income_statement_df['fiscal_quarter_number'] = financials_df['fiscal_quarter_number']
            balance_sheet_df['fiscal_quarter_number'] = financials_df['fiscal_quarter_number']
            cash_flow_statement_df['fiscal_quarter_number'] = financials_df['fiscal_quarter_number']
            computed_df['fiscal_quarter_number'] = financials_df['fiscal_quarter_number']
            misc_df['fiscal_quarter_number'] = financials_df['fiscal_quarter_number']
             
        # Return a dictionary with the separated DataFrames
        return {
            'income_statement': income_statement_df,
            'balance_sheet': balance_sheet_df,
            'cash_flow_statement': cash_flow_statement_df,
            'computed': computed_df,
            'misc': misc_df
        }
    
    # Get ticker; default to 'AAPL' if not provided in full_data
    ticker = full_data['metadata']['symbol']
    
    # Extract metadata and financials from full_data
    company_metadata     = full_data['metadata']
    financials_annual    = full_data['financials']['annual']
    financials_quarterly = full_data['financials']['quarterly']
    
    # Create DataFrames for annual and quarterly financials with sorted columns
    financials_annual_df = pd.DataFrame(financials_annual).reindex(
        sorted(pd.DataFrame(financials_annual).columns), axis=1
    )
    financials_quarterly_df = pd.DataFrame(financials_quarterly).reindex(
        sorted(pd.DataFrame(financials_quarterly).columns), axis=1
    )
    

    
    # Create dictionaries for annual and quarterly financials
    financials_annual_dict    = create_financials_dict(financials_annual_df, client)
    financials_quarterly_dict = create_financials_dict(financials_quarterly_df, client)
    
    return {
        'ticker': ticker,
        'metadata': company_metadata,
        'financials_annual': financials_annual_dict,
        'financials_quarterly': financials_quarterly_dict
    }

def save_company_info(ticker, financials_data_annual, financials_data_quarterly, company_metadata):
    """
    Saves company financial data (annual and quarterly) and metadata to disk.
    
    This function:
      - Creates a folder for the given ticker under 'company_data' if it doesn't exist.
      - For each period ('annual' and 'quarterly'), creates a subfolder and saves each
        financial DataFrame as a CSV file.
      - Saves the company metadata as a JSON file in the ticker folder.
    
    Parameters:
      ticker (str): The company ticker.
      financials_data_annual (dict): Dictionary of annual financial DataFrames.
      financials_data_quarterly (dict): Dictionary of quarterly financial DataFrames.
      company_metadata (dict): Dictionary containing company metadata.
    """
    import os, json

    parent_folder = 'company_data'
    ticker_folder = os.path.join(parent_folder, ticker)
    if not os.path.exists(ticker_folder):
        os.makedirs(ticker_folder)
        print(f"Created folder: {ticker_folder}")

    # Function to save financial data for a given period
    def save_financials_for_period(financials_data, period):
        period_folder = os.path.join(ticker_folder, period)
        if not os.path.exists(period_folder):
            os.makedirs(period_folder)
            print(f"Created folder for {period} data: {period_folder}")

        for key, df in financials_data.items():
            file_path = os.path.join(period_folder, f'{key}.csv')
            df.to_csv(file_path)
            print(f"{key} data saved to {file_path}")

    # Save annual financials
    save_financials_for_period(financials_data_annual, 'annual')

    # Save quarterly financials
    save_financials_for_period(financials_data_quarterly, 'quarterly')

    # Save company metadata to JSON
    metadata_path = os.path.join(ticker_folder, 'metadata.json')
    with open(metadata_path, 'w') as f:
        json.dump(company_metadata, f, indent=4)
    print(f"Metadata saved to {metadata_path}")

#update the log file with the latest data
def update_log(ticker_str, last_updated, latest_earnings_date, is_updated_on_server):
    parent_folder_name = 'company_data'
    
    #if log file does not exist, create it using create_log
    if not os.path.exists(os.path.join(parent_folder_name, 'log.csv')):
        create_log()
        
    #retreive the log file as a dataframe, 
    #check if the ticker exists in the log file, if it does, update the row with the new data
    #if it does not, create a new row with the new data
    log_file_path = os.path.join(parent_folder_name, 'log.csv')
    log_df = pd.read_csv(log_file_path)
    
    if ticker_str in log_df['ticker_name'].values:
        log_df.loc[log_df['ticker_name'] == ticker_str, 'last_updated'] = last_updated
        log_df.loc[log_df['ticker_name'] == ticker_str, 'latest_earnings_date'] = latest_earnings_date
        log_df.loc[log_df['ticker_name'] == ticker_str, 'is_updated_on_server'] = is_updated_on_server
    else:
        new_entry = pd.DataFrame({'ticker_name': [ticker_str], 
                                'last_updated': [last_updated], 
                                'latest_earnings_date': [latest_earnings_date], 
                                'is_updated_on_server': [is_updated_on_server]})
        log_df = pd.concat([log_df, new_entry], ignore_index=True)
    #sort the log by ticker name
    log_df = log_df.sort_values(by='ticker_name')
    log_df.to_csv(log_file_path, index=False)
    print('Log updated')

 #creates an empty log file if it does not exist

def create_log():
    parent_folder_name = 'company_data'
    # if log file does not exist, create it, it should be a csv
    # else print that it exists
    log_file_path = os.path.join(parent_folder_name, 'log.csv')
    column_names = ['ticker_name', 'last_updated', 'latest_earnings_date','is_updated_on_server']
    column_names_str = ','.join(column_names)
    
    if not os.path.exists(log_file_path):
        print('Log File doesnt exist! creating it now')
        with open(log_file_path, 'w') as f:
            f.write(column_names_str + '\n')
    else:
        print('Log File already exists')    
#------------------------------------------------------------

retrieve_data_for_companies(subindustry_companies_query, should_update=should_update)





In [ ]:
#Information for queries
usage_data = client.get_usage()['quota']
print("Data Usage:")
print(f"  Used     : {usage_data['used']}")
print(f"  Remaining: {usage_data['remaining']}")
print(f"  Resets   : {usage_data['resets']}")

income_statment_fields = retrieve_data('AAPL',statement_type='income_statement' ,should_update=should_update).columns.to_list()
balance_sheet_fields = retrieve_data('AAPL',statement_type='balance_sheet' ,should_update=should_update).columns.to_list()
cash_flow_statement_fields = retrieve_data('AAPL',statement_type='cash_flow_statement' ,should_update=should_update).columns.to_list()
computed_fields = retrieve_data('AAPL',statement_type='computed' ,should_update=should_update).columns.to_list()
misc_fields = retrieve_data('AAPL',statement_type='misc' ,should_update=should_update).columns.to_list()

#neatly print the fields
print('Income Statment Fields:')
display(income_statment_fields)
print('\n')
print('Balance Sheet Fields:')
display(balance_sheet_fields)
print('\n')
print('Cash Flow Statement Fields:')
display(cash_flow_statement_fields)
print('\n')
print('Computed Fields:')
display(computed_fields)
print('\n')
print('Misc Fields:')
display(misc_fields)
print('\n')



In [ ]:
#plot the p/e, p/b, p/fcf of the company against the subindustry median
#define plt
import plotly.express as px
def get_metric_df_from_companies(companies, metric_key, data_type='quarterly', 
                                 statement_type='computed', should_update=False):
    """
    Retrieves a DataFrame for a given metric (column from data) across multiple companies.

    Parameters:
        companies (list): List of company identifiers in the format 'TICKER:COUNTRY'
        metric_key (str): The key in the data dict to extract (e.g., 'revenue_growth', 'price_to_earnings')
        data_type (str): Data frequency, default is 'quarterly'
        statement_type (str): Type of statement to retrieve, default is 'computed'
        should_update (bool): Flag whether to force update data

    Returns:
        pd.DataFrame: DataFrame with period end dates as index and each column containing the metric for a company.
    """
    import pandas as pd
    # Initialize an empty DataFrame to collect series
    metric_df = pd.DataFrame()

    for company in companies:
        # Extract ticker from the string, assumed to be in the format 'TICKER:COUNTRY'
        company_ = company.split(':')[0]
        data = retrieve_data(company_, data_type=data_type, statement_type=statement_type, 
                               should_update=should_update)
        
        # Get the period labels and the desired metric values
        if data_type == 'annual':
            index_dates = data['fiscal_year_key']  # e.g., ['FY2002', 'FY2003', ...]
        else:
            index_dates = data['fiscal_quarter_key']  # e.g., ['FY2002.FQ1', 'FY2002.FQ2', ...]
        metric_series = data[metric_key]            # e.g., [12.34, 23.45, ..., 34.56, 45.67]
        print('company:', company_)
        print(metric_series)
        
        # Set the index of the metric series to the period labels and rename it to the company ticker
        metric_series.index = index_dates
        metric_series = metric_series.rename(company_)
        
        # Concatenate the new Series to the DataFrame aligning by the index (period labels)
        metric_df = pd.concat([metric_df, metric_series], axis=1)

    # Optionally, sort the resulting DataFrame by the index
    metric_df.sort_index(inplace=True)
    return metric_df

# Example usage:
# Define the metric key you want to plot; e.g., 'revenue_growth' or 'price_to_earnings'
data_type = 'quarterly'
statement_type = 'computed'
ticker_ = ticker

# Retrieve the metric DataFrames for the sector 
#------------------------------------------------------------
price_to_earnings_sector_df = get_metric_df_from_companies(sector_companies_query, 'price_to_earnings',
                                           data_type=data_type, statement_type=statement_type, 
                                           should_update=should_update)

price_to_earnings_growth_sector_df = get_metric_df_from_companies(sector_companies_query, 'peg_ratio',
                                           data_type=data_type, statement_type=statement_type, 
                                           should_update=should_update)

price_to_book_sector_df = get_metric_df_from_companies(sector_companies_query, 'price_to_book', 
                                             data_type=data_type, statement_type=statement_type, 
                                             should_update=should_update)


price_to_fcf_sector_df = get_metric_df_from_companies(sector_companies_query, 'price_to_fcf',
                                           data_type=data_type, statement_type=statement_type, 
                                           should_update=should_update)

#------------------------------------------------------------ 





# Retrieve the metric DataFrames for the subindustry
#------------------------------------------------------------
price_to_earnings_sub_industry_df = get_metric_df_from_companies(subindustry_companies_query, 'price_to_earnings', 
                                           data_type=data_type, statement_type=statement_type, 
                                           should_update=should_update)

price_to_earnings_growth_sub_industry_df = get_metric_df_from_companies(subindustry_companies_query, 'peg_ratio', 
                                           data_type=data_type, statement_type=statement_type, 
                                           should_update=should_update)

price_to_book_sub_industry_df = get_metric_df_from_companies(subindustry_companies_query, 'price_to_book', 
                                           data_type=data_type, statement_type=statement_type, 
                                           should_update=should_update)

price_to_fcf_sub_industry_df = get_metric_df_from_companies(subindustry_companies_query, 'price_to_fcf', 
                                           data_type=data_type, statement_type=statement_type, 
                                           should_update=should_update)
#------------------------------------------------------------


    
common_index = price_to_earnings_sector_df.index.union(price_to_book_sector_df.index).union(price_to_fcf_sector_df.index)

price_to_earnings_sector_df = price_to_earnings_sector_df.reindex(common_index)
price_to_earnings_growth_sector_df = price_to_earnings_growth_sector_df.reindex(common_index)
price_to_book_sector_df = price_to_book_sector_df.reindex(common_index)
price_to_fcf_sector_df = price_to_fcf_sector_df.reindex(common_index)

pe_sector_median   = price_to_earnings_sector_df.median(axis=1)
pe_growth_sector_median = price_to_earnings_growth_sector_df.median(axis=1)
pb_sector_median   = price_to_book_sector_df.median(axis=1)
pfcf_sector_median = price_to_fcf_sector_df.median(axis=1)


common_index = price_to_earnings_sub_industry_df.index.union(price_to_book_sub_industry_df.index).union(price_to_fcf_sub_industry_df.index)

price_to_earnings_sub_industry_df = price_to_earnings_sub_industry_df.reindex(common_index)
price_to_earnings_growth_sub_industry_df = price_to_earnings_growth_sub_industry_df.reindex(common_index)
price_to_book_sub_industry_df = price_to_book_sub_industry_df.reindex(common_index)
price_to_fcf_sub_industry_df = price_to_fcf_sub_industry_df.reindex(common_index)


pe_subindustry_median   = price_to_earnings_sub_industry_df.median(axis=1)
pe_growth_subindustry_median = price_to_earnings_growth_sub_industry_df.median(axis=1)
pb_subindustry_median   = price_to_book_sub_industry_df.median(axis=1)
pfcf_subindustry_median = price_to_fcf_sub_industry_df.median(axis=1)



company_price_to_earnings = price_to_earnings_sub_industry_df[ticker_]
company_price_to_earnings_growth = price_to_earnings_growth_sub_industry_df[ticker_]
company_price_to_book = price_to_book_sub_industry_df[ticker_]
company_price_to_fcf = price_to_fcf_sub_industry_df[ticker_]



print('Price to Earnings Median')
display(pe_subindustry_median)

from plotly.subplots import make_subplots
import plotly.graph_objects as go
# plot company metrics against the subindustry median and their differences

def plot_metric_comparison(metric_name, company_series, subindustry_series, ticker, last_periods=40):
    """
    Plots a metric for a company against the subindustry median over the last `last_periods` periods 
    on the left, and the difference (company - subindustry) on the right with a horizontal zero line.
    """
    periods = subindustry_series.index[-last_periods:]
    subindustry_values = subindustry_series[-last_periods:]
    company_values = company_series[-last_periods:]
    
    metric_diff = company_series - subindustry_series
    diff_periods = metric_diff.index[-last_periods:]
    diff_values = metric_diff[-last_periods:]
    
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=[f'{ticker} {metric_name} vs Subindustry Median',
                                        f'{ticker} {metric_name} Difference (Company - Subindustry)'])
    
    fig.add_trace(
        go.Scatter(x=periods, y=subindustry_values, mode='lines', name='Subindustry Median'),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=periods, y=company_values, mode='lines', name=ticker),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=diff_periods, y=diff_values, mode='lines', name='Difference'),
        row=1, col=2
    )
    fig.add_shape(type="line",
                  xref="x2", yref="y2",
                  x0=diff_periods[0], x1=diff_periods[-1],
                  y0=0, y1=0,
                  line=dict(color="gray", dash="dash"),
                  row=1, col=2)
    
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text=metric_name, row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=2)
    fig.update_yaxes(title_text="Difference", row=1, col=2)
    
    fig.update_layout(title_text=f'{ticker} {metric_name} Comparison and Difference', showlegend=True)
    fig.show()








#Plot Sector P/E vs Subindustry P/E, in one plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=pe_sector_median.index, y=pe_sector_median, mode='lines',
                         name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=pe_subindustry_median.index, y=pe_subindustry_median, mode='lines',
                         name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    template="plotly_dark",
    title=f'{sector} Sector vs {subindustry} Subindustry Price to Earnings Median',
    xaxis_title='Date',
    yaxis_title='Price to Earnings'
)
fig.show()

# Plot Price to Earnings
plot_metric_comparison(
    metric_name='Price to Earnings',
    company_series=company_price_to_earnings,
    subindustry_series=pe_subindustry_median,
    ticker=ticker_
)

# Plot Sector P/E Growth vs Subindustry P/E Growth, in one plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=pe_growth_sector_median.index, y=pe_growth_sector_median, mode='lines',
                         name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=pe_growth_subindustry_median.index, y=pe_growth_subindustry_median, mode='lines',
                         name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Price to Earnings Growth Median',
    xaxis_title='Date',
    yaxis_title='Price to Earnings Growth',
    template="plotly_dark"
)
fig.show()


# Plot Price to Earnings Growth
plot_metric_comparison(
    metric_name='Price to Earnings Growth',
    company_series=company_price_to_earnings_growth,
    subindustry_series=pe_growth_subindustry_median,
    ticker=ticker_
)



# Plot Sector P/B vs Subindustry P/B, in one plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=pb_sector_median.index, y=pb_sector_median, mode='lines', 
                         name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=pb_subindustry_median.index, y=pb_subindustry_median, mode='lines', 
                         name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Price to Book Median',
    template="plotly_dark",
    xaxis_title='Date',
    yaxis_title='Price to Book'
)
fig.show()


# Plot Price to Book
plot_metric_comparison(
    metric_name='Price to Book',
    company_series=company_price_to_book,
    subindustry_series=pb_subindustry_median,
    ticker=ticker_
)


# Plot Sector P/FCF vs Subindustry P/FCF, in one plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=pfcf_sector_median.index, y=pfcf_sector_median, mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=pfcf_subindustry_median.index, y=pfcf_subindustry_median, mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Price to FCF Median',
    xaxis_title='Date',
    yaxis_title='Price to FCF',
    template="plotly_dark"
)
fig.show()

# Plot Price to FCF
plot_metric_comparison(
    metric_name='Price to FCF',
    company_series=company_price_to_fcf,
    subindustry_series=pfcf_subindustry_median,
    ticker=ticker_
)





In [ ]:
import pandas_datareader.data as web
import yfinance as yf
import datetime
import pandas as pd
import statsmodels.api as sm

# Function to simplify datetime index format


# Define the stock ticker
ticker = "SPY"  # You can change to your stock of choice

# Define period and interval
period = 'max'  # Maximum period to get historical data
interval = '1d'

# Fetch stock's historical closing prices
ticker_close = yf.Ticker(ticker).history(period=period, interval=interval)['Close']
ticker_close = simplify_datetime_index(ticker_close)

# Define start and end dates for Fama-French data
start = datetime.datetime(1963, 1, 1)  # Daily data is available from 1963 onward
end = datetime.datetime.today()

# Fetch **daily** Fama-French 5-factor data
ff5_daily = web.DataReader('F-F_Research_Data_5_Factors_2x3_Daily', 'famafrench', start, end)

# Convert Fama-French data to pandas datetime
ff5_daily = ff5_daily[0].reset_index()
ff5_daily['Date'] = pd.to_datetime(ff5_daily['Date'], format='%Y%m%d')
ff5_daily = ff5_daily.set_index('Date')

# Align the data (Ensure both stock and Fama-French data have the same date range)
common_index = ff5_daily.index.intersection(ticker_close.index)
ff5_daily = ff5_daily.loc[common_index]
ticker_close = ticker_close.loc[common_index]

# Convert daily data to monthly data (resample to monthly frequency)
ff5_monthly = ff5_daily.resample('M').last()
ticker_close_monthly = ticker_close.resample('M').last()

# Calculate monthly returns for the stock (percentage change)
ticker_returns = ticker_close_monthly.pct_change().dropna()

# Calculate excess returns (stock return - risk free rate)
rf = ff5_monthly['RF'] / 100  # Convert risk-free rate from percentage to decimal
excess_stock_returns = ticker_returns - rf

# Ensure data is aligned by date (final check for common dates between stock returns and Fama-French data)
common_dates = ticker_returns.index.intersection(ff5_monthly.index)
print(f"Found {len(common_dates)} common dates between stock returns and Fama-French data")

# Align the datasets (both Fama-French and excess returns)
ff5_monthly = ff5_monthly.loc[common_dates]
excess_stock_returns = excess_stock_returns.loc[common_dates]

# Prepare for the Fama-French 5-factor regression
X = ff5_monthly[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']].copy() / 100  # Convert from percentage to decimal
X = sm.add_constant(X)  # Add constant for alpha (intercept term)
y = excess_stock_returns

# Print shapes of X and y to check if they align properly
print(f"Shape of X (Fama-French factors): {X.shape}")
print(f"Shape of y (Excess stock returns): {y.shape}")

# Perform the OLS regression
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Get the regression results
print(results.summary())


In [ ]:
# Income Statement Analysis

from plotly.subplots import make_subplots


revenue_sector = get_metric_df_from_companies(sector_companies_query, 'revenue', data_type='quarterly', statement_type='income_statement', should_update=should_update).fillna(method='ffill').fillna(method='bfill')
revenue_cagr_10_sector = get_metric_df_from_companies(sector_companies_query, 'revenue_cagr_10', data_type='quarterly', statement_type='computed', should_update=should_update)
revenue_growth_sector = get_metric_df_from_companies(sector_companies_query, 'revenue_growth', data_type='quarterly', statement_type='computed', should_update=should_update)
revenue_yoy_sector = revenue_sector.pct_change(4) * 100
revenue_qoq_sector = revenue_sector.pct_change(1) * 100
revenue_cagr_sector = (revenue_sector.iloc[-1] / revenue_sector.iloc[0]) ** (1 / (len(revenue_sector) - 1)) - 1


revenue_sub_industry = get_metric_df_from_companies(subindustry_companies_query, 'revenue', data_type='quarterly', statement_type='income_statement', should_update=should_update)
revenue_cagr_10_sub_industry = get_metric_df_from_companies(subindustry_companies_query, 'revenue_cagr_10', data_type='quarterly', statement_type='computed', should_update=should_update)
revenue_growth_sub_industry = get_metric_df_from_companies(subindustry_companies_query, 'revenue_growth', data_type='quarterly', statement_type='computed', should_update=should_update)
revenue_yoy_sub_industry = revenue_sub_industry.pct_change(4) * 100
revenue_qoq_sub_industry = revenue_sub_industry.pct_change(1) * 100
revenue_cagr_sub_industry = (revenue_sub_industry.iloc[-1] / revenue_sub_industry.iloc[0]) ** (1 / (len(revenue_sub_industry) - 1)) - 1 



#plot revenue year over year

# Plot Revenue YoY with industry median for comparison on the left 
# and the difference (YoY revenue - industry median) on the right

import plotly.graph_objects as go

sub_industry_median_rev_yoy = revenue_yoy_sub_industry.median(axis=1)
sector_median_rev_yoy = revenue_yoy_sector.median(axis=1)


fig = go.Figure()
fig.add_trace(go.Scatter(x=revenue_yoy_sector.index, y=revenue_yoy_sector.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=revenue_yoy_sub_industry.index, y=revenue_yoy_sub_industry.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Revenue YoY Median',
    xaxis_title='Date',
    yaxis_title='Revenue YoY',
    template="plotly_dark"
)
fig.show()


rev_diff = revenue_yoy_sub_industry[ticker_] - sub_industry_median_rev_yoy
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=[f'Revenue YoY Comparison: {ticker_} vs Industry Median', 'Revenue YoY Difference (Company - Industry)']
)

# Left subplot: Plot company YoY revenue and industry median
fig.add_trace(
    go.Scatter(x=revenue_yoy_sub_industry.index, y=revenue_yoy_sub_industry[ticker_], mode='lines',
               name=f'Revenue YoY ({ticker_})'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=revenue_yoy_sub_industry.index, y=sub_industry_median_rev_yoy, mode='lines',
               name='Industry Median Revenue YoY'),
    row=1, col=1
)
# Add horizontal line at 0 for left subplot
fig.add_shape(
    type="line",
    xref="x1", yref="y1",
    x0=revenue_yoy_sub_industry.index[0], x1=revenue_yoy_sub_industry.index[-1],
    y0=0, y1=0,
    line=dict(dash="dash", color="gray"),
    row=1, col=1
)

# Right subplot: Plot the difference
fig.add_trace(
    go.Scatter(x=revenue_yoy_sub_industry.index, y=rev_diff, mode='lines', name='YoY Revenue Difference'),
    row=1, col=2
)
# Add horizontal line at 0 for right subplot
fig.add_shape(
    type="line",
    xref="x2", yref="y2",
    x0=revenue_yoy_sub_industry.index[0], x1=revenue_yoy_sub_industry.index[-1],
    y0=0, y1=0,
    line=dict(dash="dash", color="gray"),
    row=1, col=2
)

fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_yaxes(title_text="Growth Rate (%)", row=1, col=1)
fig.update_xaxes(title_text="Date", row=1, col=2)
fig.update_yaxes(title_text="Difference (%)", row=1, col=2)

fig.update_layout(title_text="Revenue Year-over-Year Growth and Difference", showlegend=True)
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=revenue_qoq_sector.index, y=revenue_qoq_sector.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=revenue_qoq_sub_industry.index, y=revenue_qoq_sub_industry.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Revenue QoQ Median',
    xaxis_title='Date',
    yaxis_title='Revenue QoQ',
    template="plotly_dark"
)
fig.show()


sub_industry_industry_median_rev_qoq = revenue_qoq_sub_industry.median(axis=1)

fig = go.Figure()

fig.add_trace(go.Scatter(x=revenue_qoq_sub_industry.index, y=revenue_qoq_sub_industry[ticker_], mode='lines', name='Revenue QoQ'))
fig.add_trace(go.Scatter(x=revenue_qoq_sub_industry.index, y=sub_industry_industry_median_rev_qoq, mode='lines', name='Industry Median Revenue QoQ'))

fig.add_hline(y=0, line_dash='dash', line_color='gray')

fig.update_layout(title='Revenue Quarter-over-Quarter Growth', xaxis_title='Date', yaxis_title='Growth Rate (%)')
fig.show()


display(revenue_cagr_sector)
'''
fig = go.Figure()
fig.add_trace(go.Scatter(x=revenue_cagr_sector.index, y=revenue_cagr_sector.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=revenue_cagr_sub_industry.index, y=revenue_cagr_sub_industry.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Revenue CAGR Median',
    xaxis_title='Date',
    yaxis_title='Revenue CAGR',
    template="plotly_dark"
)
fig.show()

'''
# Plot revenue_cagr_10 for the company along with the industry median

# Compute the industry median for revenue_cagr_10
sub_industry_industry_median_rev_cagr_10 = revenue_cagr_10_sub_industry.median(axis=1)

# Create a figure with Plotly
fig = go.Figure()

# Add the company's revenue CAGR (10) trace
fig.add_trace(go.Scatter(
    x=revenue_cagr_10_sub_industry.index,
    y=revenue_cagr_10_sub_industry[ticker_],
    mode='lines',
    name=f'{ticker_} Revenue CAGR (10)'
))

# Add the industry median trace
fig.add_trace(go.Scatter(
    x=revenue_cagr_10_sub_industry.index,
    y=sub_industry_industry_median_rev_cagr_10,
    mode='lines',
    name='Industry Median Revenue CAGR (10)'
))

# Optional: add a horizontal line at y=0 for reference
fig.add_hline(y=0, line_dash='dash', line_color='gray')

# Update layout, titles, and axes labels
fig.update_layout(
    title='Revenue CAGR (10) vs Industry Median',
    xaxis_title='Date',
    yaxis_title='Revenue CAGR (10)',
    showlegend=True
)

# Display the figure
fig.show()





In [ ]:
# Balance Sheet Analysis

In [ ]:
# Cash Flow Statement Analysis

In [ ]:
# Metrics: Profitability

from plotly.subplots import make_subplots

gross_profit_margin_sector_df = get_metric_df_from_companies(sector_companies_query, 'gross_margin', data_type='quarterly', statement_type='computed', should_update=should_update)
operating_margin_sector_df = get_metric_df_from_companies(sector_companies_query, 'operating_margin', data_type='quarterly', statement_type='computed', should_update=should_update)
net_profit_margin_sector_df = get_metric_df_from_companies(sector_companies_query, 'net_income_margin', data_type='quarterly', statement_type='computed', should_update=should_update)
return_on_assets_sector_df = get_metric_df_from_companies(sector_companies_query, 'roa', data_type='quarterly', statement_type='computed', should_update=should_update)
return_on_equity_sector_df = get_metric_df_from_companies(sector_companies_query, 'roe', data_type='quarterly', statement_type='computed', should_update=should_update)
return_on_invested_capital_sector_df = get_metric_df_from_companies(sector_companies_query, 'roic', data_type='quarterly', statement_type='computed', should_update=should_update)


gross_profit_margin_sub_industry_df = get_metric_df_from_companies(subindustry_companies_query, 'gross_margin', data_type='quarterly', statement_type='computed', should_update=should_update)
operating_margin_sub_industry_df = get_metric_df_from_companies(subindustry_companies_query, 'operating_margin', data_type='quarterly', statement_type='computed', should_update=should_update)
net_profit_margin_sub_industry_df = get_metric_df_from_companies(subindustry_companies_query, 'net_income_margin', data_type='quarterly', statement_type='computed', should_update=should_update)
return_on_assets_sub_industry_df = get_metric_df_from_companies(subindustry_companies_query, 'roa', data_type='quarterly', statement_type='computed', should_update=should_update)
return_on_equity_sub_industry_df = get_metric_df_from_companies(subindustry_companies_query, 'roe', data_type='quarterly', statement_type='computed', should_update=should_update)
return_on_invested_capital_sub_industry_df = get_metric_df_from_companies(subindustry_companies_query, 'roic', data_type='quarterly', statement_type='computed', should_update=should_update)

gross_profit_margin_sub_industry_ticker = gross_profit_margin_sub_industry_df[ticker_]
operating_margin_sub_industry_ticker = operating_margin_sub_industry_df[ticker_]
net_profit_margin_sub_industry_ticker = net_profit_margin_sub_industry_df[ticker_]
return_on_assets_sub_industry_ticker = return_on_assets_sub_industry_df[ticker_]
return_on_equity_sub_industry_ticker = return_on_equity_sub_industry_df[ticker_]
return_on_invested_capital_sub_industry_ticker = return_on_invested_capital_sub_industry_df[ticker_]

import plotly.graph_objects as go

def plot_metric_with_median(metric_df, ticker_series, metric_name, last_n=40, horizontal_line_value=0):
    """
    Plots a metric for a given ticker against its subindustry median over the last `last_n` periods.
    Adds a horizontal line at the specified `horizontal_line_value` and vertical (light grey) lines at
    the start of each new fiscal year. Assumes that the ticker_series index is a list of strings in a 
    non-standard format (e.g., "FY2002.FQ1") and uses a custom extraction of the year.

    Parameters:
        metric_df (pd.DataFrame): DataFrame containing the metric data for all companies.
        ticker_series (pd.Series): Series for the ticker being plotted (index as strings).
        metric_name (str): Name of the metric (e.g., 'Price to Earnings').
        last_n (int): Number of recent periods to plot.
        horizontal_line_value (float): The y-value at which to add the horizontal line.
    """
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    # Calculate subindustry median from the metric DataFrame
    median_series = metric_df.median(axis=1)
    
    # Slice the last_n entries (x-axis values remain as strings)
    ts_recent = ticker_series[-last_n:]
    median_recent = median_series[-last_n:]
    
    # Create subplots: Left (ticker metric) and Right (ticker metric vs subindustry median)
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=(f'{ticker_series.name} {metric_name}',
                                        f'{ticker_series.name} {metric_name} vs Subindustry Median'))
    
    # Left subplot: Ticker metric with horizontal line
    fig.add_trace(
        go.Scatter(x=list(ts_recent.index),
                   y=list(ts_recent),
                   mode='lines',
                   name=ticker_series.name),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=[ts_recent.index[0], ts_recent.index[-1]],
                   y=[horizontal_line_value, horizontal_line_value],
                   mode='lines',
                   line=dict(dash='dash', color='gray'),
                   showlegend=False),
        row=1, col=1
    )
    
    # Right subplot: Subindustry median and ticker metric with horizontal line
    fig.add_trace(
        go.Scatter(x=list(median_recent.index),
                   y=list(median_recent),
                   mode='lines',
                   name='Subindustry Median'),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(x=list(ts_recent.index),
                   y=list(ts_recent),
                   mode='lines',
                   name=ticker_series.name),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(x=[ts_recent.index[0], ts_recent.index[-1]],
                   y=[horizontal_line_value, horizontal_line_value],
                   mode='lines',
                   line=dict(dash='dash', color='gray'),
                   showlegend=False),
        row=1, col=2
    )
    
    # Custom extraction: assume index strings follow the pattern "FYXXXX.FQY"
    # Extract the fiscal year portion manually.
    x_categories = list(ts_recent.index)
    # This extraction assumes the year is from character position 2 to 6:
    years = [s[2:6] for s in x_categories if len(s) >= 6]
    
    # Find the category strings where a new year starts
    year_start_positions = []
    if years:
        current_year = years[0]
        # The first entry is always the start of the first year
        year_start_positions.append(x_categories[0])
        for i, s in enumerate(x_categories):
            # Check if year changes by extracting the year portion from each string
            year = s[2:6] if len(s) >= 6 else None
            if year and year != current_year:
                year_start_positions.append(s)
                current_year = year

    # Add vertical lines (using category values) at the beginning of each new fiscal year
    for x_val in year_start_positions:
        fig.add_vline(x=x_val, line=dict(color='lightgrey' ), row=1, col=1)
        fig.add_vline(x=x_val, line=dict(color='lightgrey'), row=1, col=2)

    # Update axes labels and layout
    fig.update_xaxes(title_text="Period", row=1, col=1)
    fig.update_yaxes(title_text=metric_name, row=1, col=1)
    fig.update_xaxes(title_text="Period", row=1, col=2)
    fig.update_yaxes(title_text=metric_name, row=1, col=2)
    
    fig.update_layout(title_text=f'{ticker_series.name} {metric_name}', showlegend=True)
    fig.show()
    # Update layout and axes
    fig.update_layout(title_text=f'{ticker_series.name} {metric_name}', showlegend=True)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text=metric_name, row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=2)
    fig.update_yaxes(title_text=metric_name, row=1, col=2)
    return fig
    
 

# plot Sector gross profit margin median vs Subindustry gross profit margin median
fig = go.Figure()
fig.add_trace(go.Scatter(x=gross_profit_margin_sector_df.index, y=gross_profit_margin_sector_df.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=gross_profit_margin_sub_industry_df.index, y=gross_profit_margin_sub_industry_df.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Gross Profit Margin Median',
    xaxis_title='Date',
    yaxis_title='Gross Profit Margin',
    template="plotly_dark"
)
fig.show()

plot_metric_with_median(gross_profit_margin_sub_industry_df, gross_profit_margin_sub_industry_ticker, 'Gross Profit Margin')

fig = go.Figure()
fig.add_trace(go.Scatter(x=operating_margin_sector_df.index, y=operating_margin_sector_df.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=operating_margin_sub_industry_df.index, y=operating_margin_sub_industry_df.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Operating Profit Margin Median',
    xaxis_title='Date',
    yaxis_title='Operating Profit Margin',
    template="plotly_dark"
)
fig.show()

plot_metric_with_median(operating_margin_sub_industry_df, operating_margin_sub_industry_ticker, 'Operating Profit Margin')

fig = go.Figure()
fig.add_trace(go.Scatter(x=net_profit_margin_sector_df.index, y=net_profit_margin_sector_df.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=net_profit_margin_sub_industry_df.index, y=net_profit_margin_sub_industry_df.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Net Profit Margin Median',
    xaxis_title='Date',
    yaxis_title='Net Profit Margin',
    template="plotly_dark"
)
fig.show()

plot_metric_with_median(net_profit_margin_sub_industry_df, net_profit_margin_sub_industry_ticker, 'Net Profit Margin')

fig = go.Figure()
fig.add_trace(go.Scatter(x=return_on_assets_sector_df.index, y=return_on_assets_sector_df.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=return_on_assets_sub_industry_df.index, y=return_on_assets_sub_industry_df.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Return on Assets Median',
    xaxis_title='Date',
    yaxis_title='Return on Assets',
    template="plotly_dark"
)
fig.show()

plot_metric_with_median(return_on_assets_sub_industry_df, return_on_assets_sub_industry_ticker, 'Return on Assets')

fig = go.Figure()
fig.add_trace(go.Scatter(x=return_on_equity_sector_df.index, y=return_on_equity_sector_df.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=return_on_equity_sub_industry_df.index, y=return_on_equity_sub_industry_df.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Return on Equity Median',
    xaxis_title='Date',
    yaxis_title='Return on Equity',
    template="plotly_dark"
)
fig.show()

plot_metric_with_median(return_on_equity_sub_industry_df, return_on_equity_sub_industry_ticker, 'Return on Equity')

fig = go.Figure()
fig.add_trace(go.Scatter(x=return_on_invested_capital_sector_df.index, y=return_on_invested_capital_sector_df.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=return_on_invested_capital_sub_industry_df.index, y=return_on_invested_capital_sub_industry_df.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Return on Invested Capital Median',
    xaxis_title='Date',
    yaxis_title='Return on Invested Capital',
    template="plotly_dark"
)
fig.show()

plot_metric_with_median(return_on_invested_capital_sub_industry_df, return_on_invested_capital_sub_industry_ticker, 'Return on Invested Capital')





In [ ]:
# Metrics: Liquidity

current_assets_sector = get_metric_df_from_companies(sector_companies_query, 'total_current_assets', data_type='quarterly', statement_type='balance_sheet', should_update=should_update)    
current_liabilities_sector = get_metric_df_from_companies(sector_companies_query, 'total_current_liabilities', data_type='quarterly', statement_type='balance_sheet', should_update=should_update)
cash_and_cash_equivalents_sector = get_metric_df_from_companies(sector_companies_query,  'cash_and_equiv', data_type='quarterly', statement_type='balance_sheet', should_update=should_update)
inventory_sector = get_metric_df_from_companies(sector_companies_query, 'inventories', data_type='quarterly', statement_type='balance_sheet', should_update=should_update)

current_ratio_sector_df = get_metric_df_from_companies(sector_companies_query, 'current_ratio', data_type='quarterly', statement_type='computed', should_update=should_update)
quick_ratio_sector_df = (current_assets_sector - inventory_sector) / current_liabilities_sector
#cash_ratio_df = cash_and_cash_equivalents / current_liabilities


current_assets_sub_industry   = get_metric_df_from_companies(subindustry_companies_query, 'total_current_assets', data_type='quarterly', statement_type='balance_sheet', should_update=should_update)
current_liabilities_sub_industry = get_metric_df_from_companies(subindustry_companies_query, 'total_current_liabilities', data_type='quarterly', statement_type='balance_sheet', should_update=should_update)
cash_and_cash_equivalents_sub_industry = get_metric_df_from_companies(subindustry_companies_query,  'cash_and_equiv', data_type='quarterly', statement_type='balance_sheet', should_update=should_update)
inventory_sub_industry = get_metric_df_from_companies(subindustry_companies_query, 'inventories', data_type='quarterly', statement_type='balance_sheet', should_update=should_update)


current_ratio_sub_industry_df = get_metric_df_from_companies(subindustry_companies_query, 'current_ratio', data_type='quarterly', statement_type='computed', should_update=should_update)
quick_ratio_sub_industry_df = (current_assets_sub_industry - inventory_sub_industry) / current_liabilities_sub_industry
#cash_ratio_df = cash_and_cash_equivalents / current_liabilities

fig = go.Figure()
fig.add_trace(go.Scatter(x=current_ratio_sector_df.index, y=current_ratio_sector_df.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=current_ratio_sub_industry_df.index, y=current_ratio_sub_industry_df.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Current Ratio Median',
    xaxis_title='Date',
    yaxis_title='Current Ratio',
    template="plotly_dark"
)
fig.show()

plot_metric_with_median(current_ratio_sub_industry_df, current_ratio_sub_industry_df[ticker_], 'Current Ratio')

fig = go.Figure()
fig.add_trace(go.Scatter(x=quick_ratio_sector_df.index, y=quick_ratio_sector_df.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=quick_ratio_sub_industry_df.index, y=quick_ratio_sub_industry_df.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Quick Ratio Median',
    xaxis_title='Date',
    yaxis_title='Quick Ratio',
    template="plotly_dark"
)
fig.show()

plot_metric_with_median(quick_ratio_sub_industry_df, quick_ratio_sub_industry_df[ticker_], 'Quick Ratio')

#plot the quick ratio and current ratio on the same graph using plotly

fig = go.Figure()

fig.add_trace(go.Scatter(x=current_ratio_sub_industry_df.index, y=current_ratio_sub_industry_df[ticker_], mode='lines', name='Current Ratio'))
fig.add_trace(go.Scatter(x=quick_ratio_sub_industry_df.index, y=quick_ratio_sub_industry_df[ticker_], mode='lines', name='Quick Ratio'))

fig.update_layout(title='Current and Quick Ratios', xaxis_title='Date', yaxis_title='Ratio')
fig.show()


In [ ]:
# Metrics: Leverage

operating_income_sector = get_metric_df_from_companies(sector_companies_query, 'operating_income', data_type='quarterly', statement_type='income_statement', should_update=should_update)
interest_income_sector = get_metric_df_from_companies(sector_companies_query, 'interest_income', data_type='quarterly', statement_type='income_statement', should_update=should_update)
interest_expense_sector = get_metric_df_from_companies(sector_companies_query, 'interest_expense', data_type='quarterly', statement_type='income_statement', should_update=should_update)

operating_income_sub_industry = get_metric_df_from_companies(subindustry_companies_query, 'operating_income', data_type='quarterly', statement_type='income_statement', should_update=should_update)
interest_income_sub_industry = get_metric_df_from_companies(subindustry_companies_query, 'interest_income', data_type='quarterly', statement_type='income_statement', should_update=should_update)   
interest_expense_sub_industry = get_metric_df_from_companies(subindustry_companies_query, 'interest_expense', data_type='quarterly', statement_type='income_statement', should_update=should_update)

# EBIT = Operating Income + Interest Income - Interest Expense

ebit_sector = operating_income_sector + interest_income_sector - interest_expense_sector
ebit_sub_industry = operating_income_sub_industry + interest_income_sub_industry - interest_expense_sub_industry

debt_to_equity_sector_df = get_metric_df_from_companies(sector_companies_query, 'debt_to_equity', data_type='quarterly', statement_type='computed', should_update=should_update)
debt_to_assets_sector_df = get_metric_df_from_companies(sector_companies_query, 'debt_to_assets', data_type='quarterly', statement_type='computed', should_update=should_update)
interest_coverage_sector_df = ebit_sector / interest_expense_sector

debt_to_equity_sub_industry_df = get_metric_df_from_companies(subindustry_companies_query, 'debt_to_equity', data_type='quarterly', statement_type='computed', should_update=should_update)
debt_to_assets_sub_industry_df = get_metric_df_from_companies(subindustry_companies_query, 'debt_to_assets', data_type='quarterly', statement_type='computed', should_update=should_update)
interest_coverage_sub_industry_df = ebit_sub_industry / interest_expense_sub_industry


fig = go.Figure()
fig.add_trace(go.Scatter(x=debt_to_equity_sector_df.index, y=debt_to_equity_sector_df.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=debt_to_equity_sub_industry_df.index, y=debt_to_equity_sub_industry_df.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Debt to Equity Median',
    xaxis_title='Date',
    yaxis_title='Debt to Equity',
    template="plotly_dark"
)
fig.show()

plot_metric_with_median(debt_to_equity_sub_industry_df, debt_to_equity_sub_industry_df[ticker_], 'Debt to Equity',horizontal_line_value=1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=debt_to_assets_sector_df.index, y=debt_to_assets_sector_df.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=debt_to_assets_sub_industry_df.index, y=debt_to_assets_sub_industry_df.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Debt to Assets Median',
    xaxis_title='Date',
    yaxis_title='Debt to Assets',
    template="plotly_dark"
)
fig.show()

plot_metric_with_median(debt_to_assets_sub_industry_df, debt_to_assets_sub_industry_df[ticker_], 'Debt to Assets',horizontal_line_value=1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=interest_coverage_sector_df.index, y=interest_coverage_sector_df.median(axis=1), mode='lines', name=f'{sector} Sector Median'))
fig.add_trace(go.Scatter(x=interest_coverage_sub_industry_df.index, y=interest_coverage_sub_industry_df.median(axis=1), mode='lines', name=f'{subindustry} Subindustry Median'))
fig.update_layout(
    title=f'{sector} Sector vs {subindustry} Subindustry Interest Coverage Median',
    xaxis_title='Date',
    yaxis_title='Interest Coverage',
    template="plotly_dark"
)
fig.show()

plot_metric_with_median(interest_coverage_sub_industry_df, interest_coverage_sub_industry_df[ticker_], 'Interest Coverage')



In [ ]:
# Metrics: Efficiency
cogs = get_metric_df_from_companies(subindustry_companies_query, 'cogs', data_type='quarterly', statement_type='income_statement', should_update=should_update)

inventory = get_metric_df_from_companies(subindustry_companies_query, 'inventories', data_type='quarterly', statement_type='balance_sheet', should_update=should_update)
beginning_inventory = inventory.shift(1)
ending_inventory = inventory
average_inventory = (beginning_inventory + ending_inventory) / 2



#to compute inventory turnover, the formula is COGS / Average Inventory
inventory_turnover_df = cogs / average_inventory

#to compute accounts receivable turnover
#to compute asset turnover
#to compute accounts payable turnover
plot_metric_with_median(inventory_turnover_df, inventory_turnover_df[ticker_], 'Inventory Turnover')

